In [15]:
import pandas as pd, numpy as np
from copy import deepcopy
pred = pd.read_csv('./anom/TestSet2Predictions.csv')


In [2]:
snr_pred = np.load('/home/yunfanz/snr_preds.npy')
snr_pred = np.argmax(snr_pred, axis=1)
print(snr_pred.shape)

(100000,)


In [16]:
cols = pred.columns[1:]
pred.head(n=5)
print(cols, len(cols))


Index(['16PSK', '2FSK_5KHz', '2FSK_75KHz', '8PSK', 'AM_DSB', 'AM_SSB',
       'APSK16_c34', 'APSK32_c34', 'BPSK', 'CPFSK_5KHz', 'CPFSK_75KHz',
       'FM_NB', 'FM_WB', 'GFSK_5KHz', 'GFSK_75KHz', 'GMSK', 'MSK', 'NOISE',
       'OQPSK', 'PI4QPSK', 'QAM16', 'QAM32', 'QAM64', 'QPSK'],
      dtype='object') 24


In [17]:
pred['PI4QPSK'] = 1.e-15
# pred['FM_NB'] = 1.e-15  #set this next time, skipping to see effect
# pred['FM_WB'] = 1.e-15

In [5]:
#confusion = ['QAM32','QAM64','QAM16','APSK16_c34','APSK32_c34']
confusion = ['8PSK','16PSK']
#confusion = ['CPFSK_5KHz', 'CPFSK_75KHz']
con_arr = np.sort([list(cols).index(col) for col in confusion])
con_arr

array([0, 3])

In [18]:
pred_arr = np.array(pred[cols])
print(pred_arr.shape)
pred_arr[1]


(100000, 24)


array([1.89121664e-02, 1.99608970e-04, 1.44097570e-03, 8.86943415e-02,
       1.99608970e-04, 1.99608970e-04, 2.09630281e-03, 6.18937251e-04,
       1.99608970e-04, 1.99608970e-04, 6.64772757e-04, 1.99608970e-04,
       1.99608970e-04, 1.39008155e-02, 5.18228225e-02, 1.99608970e-04,
       1.99608970e-04, 1.99608970e-04, 1.10019304e-01, 1.00000000e-15,
       2.26312375e-04, 8.39863904e-04, 2.13609499e-04, 6.02118909e-01])

In [13]:
import pickle
f = open('/home/yunfanz/Test_Set_1_Army_Signal_Challenge.pkl', 'rb')
testdata = pickle.load(f, encoding='latin1')
testdata = np.stack([testdata[i] for i in range(1, len(testdata.keys())+1)], axis=0)


In [8]:
from keras.models import load_model
model = load_model('/home/yunfanz/domain0.h5')
qampred = model.predict(testdata)

Using TensorFlow backend.


In [7]:
m = [[],[],[],[],[],[]]
for j, row in enumerate(pred_arr):
    if np.argmax(row) in con_arr:
#         snr_cat = int(snr_pred[j])
#         #m[snr_cat].append(np.amax(row))
        
        group = deepcopy(row[con_arr])
        #print(group)
        for i in range(con_arr.size):
#            group[i] = 0.85*group[i] + 0.03*np.sum(row[con_arr]) 
            group[i] = 0.9*group[i] + 0.05*np.sum(row[con_arr]) 
#         print(group)
        
#         print(pred_arr[j][con_arr])
#         rowsum = np.sum(pred_arr[j][con_arr])
        pred_arr[j][con_arr] = group#rowsum*qampred[j]#group
        #print(pred_arr[j][con_arr])
        #break

In [17]:
def clip_prob(pred_a, eps):
    pred_a = np.where(pred_a<eps, eps, pred_a)
    pred_a = np.where(pred_a>1-eps, 1-eps, pred_a)
    pred_a /= np.sum(pred_a, axis=1, keepdims=True)
    return pred_a
pred_arr = clip_prob(pred_arr, eps=1.e-3)
pred_arr[:,11] = 1.e-15
pred_arr[:,12] = 1.e-15  #set this next time, skipping to see effect
pred_arr[:,19] = 1.e-15

In [42]:
for M in m:
    print(len(M), np.mean(M), np.std(M))

1190 0.127223415986 0.0276660268568
1746 0.315863076414 0.128237214223
1919 0.5664353915 0.165802574612
1871 0.700519364094 0.161521020647
1867 0.736168148767 0.166813592243
2022 0.747431489719 0.170546938513


In [19]:
new_pred = pd.DataFrame(pred_arr, columns=cols)
new_pred['Index'] = pred['Index']

In [20]:
new_pred.set_index('Index', inplace=True)
new_pred.head(n=5)

,16PSK,2FSK_5KHz,2FSK_75KHz,8PSK,AM_DSB,AM_SSB,APSK16_c34,APSK32_c34,BPSK,CPFSK_5KHz,...,GFSK_75KHz,GMSK,MSK,NOISE,OQPSK,PI4QPSK,QAM16,QAM32,QAM64,QPSK
Index,,,,,,,,,,,,,,,,,,,,,
1,0.000199,0.000199,0.000199,0.000199,0.000199,0.000199,0.000199,0.000199,0.000199,0.000199,...,0.000199,0.000199,0.000199,0.000199,0.000199,1.000000e-15,0.000199,0.000199,0.000199,0.995420
2,0.018912,0.000200,0.001441,0.088694,0.000200,0.000200,0.002096,0.000619,0.000200,0.000200,...,0.051823,0.000200,0.000200,0.000200,0.110019,1.000000e-15,0.000226,0.000840,0.000214,0.602119
3,0.066701,0.000200,0.000811,0.048065,0.000200,0.000200,0.111063,0.322560,0.002865,0.000200,...,0.001615,0.001674,0.001006,0.000200,0.059355,1.000000e-15,0.026986,0.076873,0.038015,0.174513
4,0.000199,0.000199,0.000199,0.000199,0.000199,0.000199,0.000199,0.000199,0.000199,0.000199,...,0.000199,0.995420,0.000199,0.000199,0.000199,1.000000e-15,0.000199,0.000199,0.000199,0.000199
5,0.062004,0.000199,0.000199,0.933047,0.000199,0.000199,0.000199,0.000199,0.000199,0.000199,...,0.000199,0.000199,0.000199,0.000199,0.000199,1.000000e-15,0.000199,0.000765,0.000199,0.000199


In [21]:
#new_pred.set_index('Index', inplace=True)
new_pred.to_csv('/home/yunfanz/TestSet2Predictions_dom_ens1.csv',float_format='%.5f')

In [86]:
pred_mods = np.argmax(pred_arr, axis=1)

In [30]:
np.histogram(snr_pred)

(array([16726,     0, 16774,     0, 16720,     0, 16969,     0, 16228, 16583]),
 array([ 0. ,  0.5,  1. ,  1.5,  2. ,  2.5,  3. ,  3.5,  4. ,  4.5,  5. ]))

In [96]:
np.array([p in con_arr for p in pred_mods]).nonzero()[0][0]

1

In [16]:
qampred.shape

(100000, 5)

In [17]:
con_arr.shape

(5,)